### Modul de __ funciona amb: 
- `conda install gensim -c conda-forge`

#### Comentaris de classe:
- words embedings part fonamental de models per prosesament de llenguatge LLM (chat gpt) large languaje models. 
- input en forma de text i el model respon. en base: 
    - generar la paraula que amb més probabilitat seguiria la última que hem dit. 
- ho fonamenta el word embeding. És: 
    - tenim un espai de dades i aquest espai de dades i el volem reduir amb un espai inferior.
    - un text pot tenir milers de paraules. si considerem l'espai dels textos, no ho podem determinar. 
    - fem un embedding, aquest espai el representem d'una altre manera (el fem encaixar dintte un espai que ens inventem i definim nosaltres la dimensió). 
- Ho resoldrem amb xarxes neuronals. Si tenim capes i neurones, tenim el vector, tenim el convolutional head. De 12 millons de pixels, ens inventem una manera de representar-la en un vector de 512 unitats. 
- Imatge de X pixels a través de la xarxa la convertim en un vector. 
- Dataset dels pisos, tenim una taula amb 10/15 columnes que defineixen els atributs. Imatges de pisos que pasen a un vector que sería la row del detaset. Ho hem de fer amb textos
- Agafem textos, esl tokenitzem, els fem entar a la NN i en surt un vector, que es una descripció estructurada del text que hem entrat. 
- Qualsevol text es converteix en un xurro de numeros. aixo volem que ens serveixi per comparar uns textos amb els altres. 
- no només fer la conversió, sino que també hem de mantenir la estructura de dades que hi havia (forma de dades) els que s'assemblen entre si. que l'embeding preservi les similituds. Ajustar els pesos perquè tinguin més error o menys segons lo bé que prediguin les dades d'entrada. 
- Les semblançes són en termes de la semantica del text.
- Saber quines són més semblants, vector home - vector genere = vecteor dona. 
- ---
- el corpus de text, són les paraules d'entrenament. 
- si fem una matriu /dataset a les columnes documents i a les files paraules
- posem un 0/1 si surt la paraula a cada document
- matriu de 0 i 1 que diu si una determinada paraula està en un determinat document o no. 
- si agafem una fila (una paraula), que estaba a a "X" documents. 
- i n'agafem una altre (estàra en uns altres documents). 
- són dos paraules que entre elles existeix una coorelacio dels docuemnts que apareix. 
- si apareix en un també apareix en un altre. 
- document és qualsevol cosa que tingui text (paragraf, llibre, etc)
- si busquem la corelacio.
- per tots els documents (columnes). 
- la corelació entre la paraula (fila) 250 i la 3. 
- la podem mesurar com pel document 1 1·0, pel document 2 0·1, pel document x 1·1 ··· = 2
- com més vegades es repeteix dos paraules en dos documents diferents més gran és la corelació
- si sempre que està en un document no està en l'altre serpa 1·0 o 0·1, la seva corelació sempre serà 0. 
- també podem parlar de coorelació entre documents. Si apareixen les mateixes paraules en els mateixos documents tindrem més 1·1 i voldrà dir que són més semblants. 
- paraules molt coorelacioneades vol dri que estaran relacionades semanticament (cotxe i roda) o perque formen part d'un domini que són habituals (delanter i defensa). 
- si hi han dos paraules que acostumen a anar una després de l'altre, sempre que apreixi en un document, quan apareixi en l'altre document, la següent també hi serà. això es per mesurar com de possible es que una paraula vingui a continuació d'una altre. 
- només es algo que pasaria: les paraules (jo penso que)
- molts documents amb jo, penso, que totes amb 1
- no estem parlant de ordre encara (position embeding). no que una darrere l'altre sino que apareixaren juntes. 
- ---
- algebra que a partir d'aquesta idea que fa que podem entrenar aquestes xarxes neuronals per fer el encoding (encoding i embeding) és el mateix però embeding especificament a transformar o reduir l'espai de dades. . 
- un text és un punt. o paraula és un punt. podem veure com de similars són entre si. 

### Word embeddings
Word2Vec Google 2013 C implementation

"show me your friends and I'll tell you who you are"

In [1]:
import pandas as pd

#### Load Simpsons dataset

In [2]:
df = pd.read_csv('./simpson_data/simpsons_script_lines.csv')

/tmp/ipykernel_682923/663137918.py:1: DtypeWarning: Columns (4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./simpson_data/simpsons_script_lines.csv')


In [3]:
df[['raw_character_text', 'spoken_words']].head()

,raw_character_text,spoken_words
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,Lisa Simpson,Where's Mr. Bergstrom?
2,Miss Hoover,I don't know. Although I'd sure like to talk t...
3,Lisa Simpson,That life is worth living.
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...


- make sure all text documents are type 'str'

In [4]:
df['spoken_words'] = df.spoken_words.astype('str')

#### Data preprocessing (tokenization)

- classes per fer la tokenitzacio: `tokenize` és més a baix nivell i té una configuració que pot ser que algun dia haguem de tocar. 
- si el `simple_preprocess` -> spam/ham, treure caracters, treure accents, convertir minuscules (amb català, ens interessaría que més sigui diferent a mes). 

In [5]:
from gensim.utils import simple_preprocess, tokenize

In [6]:
preprocessed_data = df.spoken_words.apply(simple_preprocess)
preprocessed_data.head(3)

0    [no, actually, it, was, little, of, both, some...
1                               [where, mr, bergstrom]
2    [don, know, although, sure, like, to, talk, to...
Name: spoken_words, dtype: object

- La tokenització de LLM va més enllà i codifica per trosos per blocs, cotxes seríen dos blocs (cotxe) i (s). Es pot fer a molts nivells de finura. 
- Conservar els interrogants del final, etc
- No hi ha necessitat de baixar al nivell de cada token sigui un caràcter. 
- si ja fa alucinació amb paraules, si ho fes amb caracters, faría paraules noves.

#### Word2Vec model

- és el model que ens farà el embdding. dos fases: 
- --- 
- constriur diccionari de paraules. 
- entrenar el model.

In [7]:
from gensim.models.word2vec import Word2Vec

In [8]:
w2v = Word2Vec(min_count = 20, window = 2, alpha = 0.03, workers = 4)

- vector_size: dimension of the word vectors
- window: max. distance between related words within a sentence
- min_count: ignore words with frequency lower than this
- alpha: learning rate
- max_vocab_size: unique words (10M words~ 1GB, prune infrequent words if not enough memory)

- ---
- n'hi ha més però els principals són: 
- vector_size: tamañ del convolutional head, dimensio del embeding. Deu tenir un valor per defecte.
- window_max: donada una frase, un text, quantes paraules considerem, per davant o per darrere (paraules endavant o endarrere) considerem que poden tenir relació amb aquella. Definim un rang que considerem lògic. 
- hi ha el sample: que està relacionat amb el negative, que treiem els dos
- min_count: les paraules que es repetiran molt i altres molt extrañes (aquestes són poc rellevants o no, perque son significatives d'aquell document). Si una apareix menys vegades que x li diguem en tot el nostre corpus, no la tindrem amb compte. 
- aqui el corpus es el dataset. 
- learning_rate: com apren. 
- a parte del min_count a més a més podem limitar el tamany màxim del vocabulari de 1000 paraules (max_vocab_size). A mida que n'hi ha que apareixen més sovint, anem eliminant les més frequents, el min count va pujant sol per quedar-nos amb el que volem

#### build vocabulary corpus

In [9]:
w2v.build_vocab(preprocessed_data, progress_per = 1000)

- li passem les dades prosesades (tokens)
- progress, si passen x coses et diu un missatge

#### train model

In [10]:
%%time
w2v.train(preprocessed_data, total_examples = w2v.corpus_count, epochs = 30)

CPU times: user 22.3 s, sys: 152 ms, total: 22.5 s
Wall time: 6.04 s


(25791803, 38288880)

#### embedded word vetor

In [11]:
w2v.wv

- atribut, word vector de'n homer, bart (paraula homer, bart).
- tenim una codificació de cada una de les paraules. 
- corelació entre paraules. 
- ubiquem paraules dins aquest espai. 
- 100 dimensions. 
- com cada una de les paraules de text han acabat codificades.

In [12]:
w2v.wv['homer'], w2v.wv['bart']

(array([ 0.24761464, -0.646954  , -0.22736843, -0.31750828, -0.9213416 ,
        -0.13310805, -0.60613203,  0.56431097,  0.4907881 ,  0.7681977 ,
         0.6250498 ,  0.6801206 ,  0.1126004 ,  0.5194235 , -0.25052863,
         0.97861755,  0.36463523, -0.00327432, -0.903615  ,  0.25540316,
         0.28269526, -0.2919843 ,  0.45271057,  0.27904296, -0.7701955 ,
        -0.19838303,  0.34900033, -0.3017807 ,  0.46909657,  0.1214712 ,
        -0.7149411 ,  0.3899995 , -0.625154  , -0.20385315, -0.10980447,
        -0.40740964,  0.29905647,  0.9517245 , -0.03083142,  0.44666475,
        -0.8710561 ,  1.128301  , -0.5629991 , -0.7861473 ,  0.3163792 ,
        -0.7864798 , -0.04792315,  0.56814575,  0.36130708,  0.11740433,
        -0.21058393,  0.08846226,  0.40662685, -1.5543088 ,  0.488568  ,
        -0.0575441 , -0.11406124, -0.2408173 ,  1.4601722 ,  0.91371834,
        -0.15842807, -0.99390143,  0.23752058, -0.8745389 , -0.10709973,
        -0.00280077,  0.87932247,  0.3540226 ,  0.2

#### word similarity (closeness in the embeddig space)

In [13]:
w2v.wv.similarity('homer', 'bart')

0.7612679

- la similitud es el cosine similitarity. 
- com més petit el angle més properes són. 
- tots els numeros estàn entre 0 i 1, les diferencies no són molt grans. 
- esta tot en un hiper cub de tamany 1. 
- això són logits també. 
- això no es un convolutional head sinó un embeading head. 
- com que es el cos(x) va de -1 a 1. 
- com més angle, més semblants, vol dir que més juntes surten en el text. Cotxe i camió, son semblants i poden no sortir juntes, però és més probable que surtin juntes, si tenim prou mostres al final hauríem de sortir juntes. 
- perque això és una heurística. 
- podem tenir una mostre de tota la documentació escrita de la humanitat i no en tenim prou perque les inteligències generatives diguin tonteries. 
- pot ser una mostra minuscula respecte infinites possibilitats que tenim realment. 

#### find most similar
- és l'objectiu: trobar les més similars a una paraula. 
- amb una altre serè de preprosesament decideixen quina va darrere l'altre. 
    - sinó posariem sempre sinonims. 

In [14]:
w2v.wv.most_similar(positive = ['homer'])

[('marge', 0.8120584487915039),
 ('bart', 0.7612677812576294),
 ('lisa', 0.6570351719856262),
 ('abe', 0.5900879502296448),
 ('grampa', 0.5653467178344727),
 ('mrs', 0.5507884621620178),
 ('homie', 0.5185571312904358),
 ('dad', 0.4895704686641693),
 ('family', 0.4702400863170624),
 ('mr', 0.45906394720077515)]

- similar: paraules que un cop fent l'embeding, aquelles paraules estàn allà aprop, això vol dir que aquestes paraules puguin apareixer conjuntament. 

#### odd-one-out (find most dissimilar within list)
- es dir que pases una llista de paraules i dius la que lliga menys. en termes de embeding, la que te menys cos similarity, la que està més lluny.

In [15]:
w2v.wv.doesnt_match(['nelson', 'bart', 'milhouse'])

'nelson'

#### analogy difference

- which word is to 'woman' as 'homer' is to 'marge' ?
- relacionat amb: 
- quina paraula es a woman, lo que homer es a marge.
- la qualitat de genere de bart és home, per tant la de lisa des dona, nena, etc
- si asociem bart a home, asociem, lisa es una dona que encara es nena. 

In [16]:
w2v.wv.most_similar(positive = ["woman", "homer"], negative = ["marge"], topn = 3)

[('man', 0.6092913150787354),
 ('child', 0.5805195569992065),
 ('bear', 0.5310059189796448)]

- which word is to 'woman' as 'bart' is to 'man'?

In [17]:
w2v.wv.most_similar(positive = ["woman", "bart"], negative = ["man"], topn = 3)

[('lisa', 0.7314830422401428),
 ('homer', 0.5757341980934143),
 ('marge', 0.572046160697937)]


- el parametre negatiu

- quan fem l'encoding (word embedding), els algoritmes es generen ells mateixos exemples negatius

- positu: dues paraules seguides en el text que li ensenyes.
- negatiu: de tot el corpus, de les dos paraules que van juntes, n'agafen una altre que no vagin juntes i li posen allà. això seria un exemple negatiu
- conjunts de dos paraules: bi-grans o n-grams (n paraules juntes)

- per cada exemple positiu, crea exemples negatius (és un parametre) són exemples que no estàn juntes. 

- la forma que tenim d'evaluar-ho és amb cross entrophi (span/ham). Mesura de loss per calcular l'error. 

- exemples positius: bi grams, grups de paraules que podem treure d'una frase que van una darrere l'altre
- per cada un positiu, se'n inventa de negatius. 

- chatgpt 2 in 60 lines of numpy

- https://jaykmody.com/blog/gpt-from-scratch/

    - al chatgpt, hem d'entendre que el word embedding (no son paraules són tokens). token = unitat semantica
    - una paraula pot tenir dos significat homes (home i plural) dos sentits semàntics (2 tokens). 